In [ ]:
from google.colab import files
files.upload()
import pandas as pd

orders = pd.read_csv("orders.csv")
orders.head()
users = pd.read_json("users.json")
users.head()
import sqlite3

conn = sqlite3.connect("food.db")
cursor = conn.cursor()
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()



Saving restaurants.sql to restaurants (1).sql
Saving users.json to users (1).json
Saving orders.csv to orders (1).csv


In [ ]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [ ]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)


In [ ]:
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [ ]:
files.download("final_food_delivery_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")


In [ ]:
gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [ ]:
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [ ]:
user_spend = df.groupby("user_id")["total_amount"].sum()
count = user_spend[user_spend > 1000].count()

count


np.int64(2544)

In [ ]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels)

df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


/tmp/ipython-input-786606277.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6–5.0,2197030.75
4.1–4.5,1960326.26
3.0–3.5,1881754.57
3.6–4.0,1717494.41


In [ ]:
df[df["membership"] == "Gold"] \
.groupby("city")["total_amount"].mean() \
.sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [ ]:
restaurant_count = df.groupby("cuisine")["restaurant_id"].nunique()
revenue = df.groupby("cuisine")["total_amount"].sum()

summary = pd.concat([restaurant_count, revenue], axis=1)
summary.columns = ["restaurant_count", "revenue"]

summary.sort_values("restaurant_count")


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [ ]:
gold_orders = df[df["membership"] == "Gold"].shape[0]
total_orders = df.shape[0]

percentage = round((gold_orders / total_orders) * 100)
percentage


50

In [ ]:
restaurant_stats = df.groupby("restaurant_name_x").agg(
    avg_order=("total_amount", "mean"),
    order_count=("order_id", "count")
)

restaurant_stats[restaurant_stats["order_count"] < 20] \
.sort_values("avg_order", ascending=False)


,avg_order,order_count
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19
...,...,...
Annapurna Tiffins Punjabi,621.828947,19
Darbar Tiffins Non-Veg,596.815556,18
Darbar Restaurant Punjabi,589.972857,14


In [ ]:
df.groupby(["membership", "cuisine"])["total_amount"].sum() \
.sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [ ]:
df["order_date"] = pd.to_datetime(df["order_date"])
df["quarter"] = df["order_date"].dt.to_period("Q")

df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)


/tmp/ipython-input-3625254343.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [ ]:
gold_orders = df[df["membership"] == "Gold"].shape[0]
gold_orders



4987

In [ ]:
hyderabad_revenue = round(
    df[df["city"] == "Hyderabad"]["total_amount"].sum()
)
hyderabad_revenue


1889367

In [ ]:
distinct_users = df["user_id"].nunique()
distinct_users


2883

In [ ]:
gold_aov = round(
    df[df["membership"] == "Gold"]["total_amount"].mean(),
    2
)
gold_aov


np.float64(797.15)

In [ ]:
high_rating_orders = df[df["rating"] >= 4.5].shape[0]
high_rating_orders


3374

In [ ]:
top_gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_gold_city


'Chennai'

In [ ]:
orders_top_city = df[
    (df["membership"] == "Gold") &
    (df["city"] == top_gold_city)
].shape[0]

orders_top_city


1337